In [6]:
import os
import numpy as np
import pandas as pd
import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go
import ipywidgets as wg
from scipy import stats
from IPython.display import display
from urllib.request import urlopen
from plotly.subplots import make_subplots

psldata = pd.ExcelFile(r'/Users/Amar/Documents/WD/projects/PSLVisualisation/data/All deliveries.xlsx')
df = pd.read_excel(psldata)

df.iloc[:,-2:] = df.iloc[:,-2:].fillna('No dismisal')

print(df)



      Match_id  Innings  Over  Ball       Batting_team      Bowling_team  \
0            1        1     0     1     Peshawar Zalmi  Islamabad United   
1            1        1     0     2     Peshawar Zalmi  Islamabad United   
2            1        1     0     3     Peshawar Zalmi  Islamabad United   
3            1        1     0     4     Peshawar Zalmi  Islamabad United   
4            1        1     0     5     Peshawar Zalmi  Islamabad United   
...        ...      ...   ...   ...                ...               ...   
5409        24        2    15     6  Quetta Gladiators    Peshawar Zalmi   
5410        24        2    15     7  Quetta Gladiators    Peshawar Zalmi   
5411        24        2    16     1  Quetta Gladiators    Peshawar Zalmi   
5412        24        2    16     2  Quetta Gladiators    Peshawar Zalmi   
5413        24        2    16     3  Quetta Gladiators    Peshawar Zalmi   

       Batsman_strike     Non_striker          Bowler  Batsman_runs  \
0     Mohammad H

In [11]:

##importing libraries

import os
import numpy as np
import pandas as pd
import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go
import ipywidgets as wg
from scipy import stats
from IPython.display import display
from urllib.request import urlopen
from plotly.subplots import make_subplots


#importing data file
psldata = pd.ExcelFile(r'/Users/Amar/Documents/WD/projects/PSLVisualisation/data/All deliveries.xlsx')
df = pd.read_excel(psldata)


##cleaning data
#Replaced Nas
df.iloc[:,-2:] = df.iloc[:,-2:].fillna('No dismisal')


# In[324]:


def wick(x):
    if x != "No dismisal":
        return 1
    else:
        return 0
df['wicket'] = df['Dismisal_kind'].apply(lambda x: wick(x))




def isSix(x):
    if x == 6:
        return 1
    else:
        return 0
    
def isFour(x):
    if x == 4:
        return 1
    else:
        return 0
df['Six'] = df['Batsman_runs'].map(lambda x: isSix(x))
df['Four'] = df['Batsman_runs'].map(lambda x: isFour(x))



#Ball-by-ball detail for each batsman

df_bat = pd.merge(df.groupby(['Batsman_strike','Match_id']).sum().reset_index()[['Batsman_strike','Match_id','Batsman_runs']], df.groupby(['Batsman_strike', 'Match_id']).count().reset_index()[['Batsman_strike','Match_id', 'Ball']].rename(columns = {'Ball': 'Balls_played'}), on = ['Batsman_strike', 'Match_id'])



##Batting datafrane
df_batAvg = pd.merge(pd.merge(df_bat.groupby('Batsman_strike').mean().reset_index()[['Batsman_strike','Batsman_runs', 'Balls_played']].rename(columns = {'Batsman_runs' : 'AvgRuns_match' , 'Balls_played': 'AvgBalls_match'}), df[['Batsman_strike', 'Batting_team']].drop_duplicates(keep = 'first'), on = 'Batsman_strike') , pd.merge(df.groupby(['Batsman_strike']).sum().reset_index()[['Batsman_strike', 'Batsman_runs', 'Six', 'Four']].rename(columns = {'Batsman_runs': 'TotalRuns', 'Six':'TotalSixes', 'Four':'TotalFours'}), df.groupby(['Batsman_strike']).count().reset_index()[['Batsman_strike' , 'Ball']].rename(columns = {'Ball': 'Balls_played'}), on ='Batsman_strike'), on = 'Batsman_strike')
                                                                                                                                     
                                                                                                                                     

df_batAvg['Runs_sq'] = 0
df_batAvg['Matches_played'] = 0


#To calculate variance
for i in range(len(df_batAvg)):
    for j in range(len(df_bat)):
        if df_batAvg.iloc[i]['Batsman_strike'] == df_bat.iloc[j]['Batsman_strike']:
            df_batAvg['Runs_sq'].iloc[i] += df_bat.iloc[j]['Batsman_runs'] **2
            df_batAvg['Matches_played'].iloc[i] += 1


#Strke rate, % boundary and Variance
df_batAvg['Strike_rate'] = df_batAvg['TotalRuns'] * 100 /df_batAvg['Balls_played']
df_batAvg['PercentageRuns_boundary'] = (df_batAvg['TotalSixes']* 6 + df_batAvg['TotalFours'] * 4) * 100/df_batAvg['TotalRuns']
df_batAvg["Var"] = (df_batAvg['Runs_sq'] - df_batAvg['Matches_played']*(df_batAvg['AvgRuns_match']**2))/(df_batAvg['Matches_played'] - 1)
df_batAvg = df_batAvg.sort_values(by = 'TotalRuns', ascending = False)


##Batsman Scatter

z = df_batAvg['Batsman_strike']
trace = go.Scatter(y = df_batAvg['Strike_rate'], x= df_batAvg['AvgRuns_match'], mode = 'markers', showlegend = False , text = df_batAvg['Batsman_strike'] + "  " + df_batAvg['Batting_team'], hoverinfo =  "text", marker = dict(size = 20, symbol = 'circle' , cmin = 0, autocolorscale = True, reversescale = True, colorbar = {'title' : {'text': 'Score variance'}}, color = df_batAvg['Var']))
data = [trace]

layout = go.Layout(paper_bgcolor = 'white',plot_bgcolor = 'white' , yaxis = dict(title = dict(text = "Strike rate" , font = dict(color ='black')) ,showgrid = True, zeroline = False, color = 'black'), xaxis = dict( title= dict( text = 'Average runs'), showgrid = False, zeroline= False), title = dict(text = "All batsman's performance - Average runs, Strike rate, Score variance"))
fig = go.Figure(data = data, layout = layout)
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.plot(fig, include_plotlyjs = False, output_type = 'div')

plotly.plot(fig)


#Batsman Histogram


# team_list = list(df_batAvg['Batting_team'].unique())

# trace0 = go.Histogram(x= df_batAvg.loc[df_batAvg['Batting_team'] == team_list[0], 'AvgRuns_match'], opacity = 0.75, name = team_list[0])
# trace1 = go.Histogram(x= df_batAvg.loc[df_batAvg['Batting_team'] == team_list[1], 'AvgRuns_match'], opacity = 0.75, name = team_list[1])
# trace2 = go.Histogram(x= df_batAvg.loc[df_batAvg['Batting_team'] == team_list[2], 'AvgRuns_match'], opacity = 0.75, name = team_list[2])
# trace3 = go.Histogram(x= df_batAvg.loc[df_batAvg['Batting_team'] == team_list[3], 'AvgRuns_match'], opacity = 0.75, name = team_list[3])
# trace4 = go.Histogram(x= df_batAvg.loc[df_batAvg['Batting_team'] == team_list[4], 'AvgRuns_match'], opacity = 0.75, name = team_list[4])

# fig4 = make_subplots(rows =3, cols = 2)
# fig4.append_trace(trace0,1,1)
# fig4.append_trace(trace1,2,1)
# fig4.append_trace(trace2,2,2)
# fig4.append_trace(trace3,3,1)
# fig4.append_trace(trace4,3,2)

# fig4.layout.title = dict(text = 'Distribution of average batsman scores')
# fig4.layout.xaxis = dict(tickmode = 'linear', dtick = 10)


# plotly.offline.plot(fig4, include_plotlyjs = False, output_type = 'div')



def runlevel(x):
    if x >= 1.25*df_batAvg['AvgRuns_match'].mean():
        return 'High'
    elif x <= 0.75 * df_batAvg['AvgRuns_match'].mean():
        return 'Low'
    else:
        return 'Average'
    
def strikelevel(x):
    if x >= 1.25*df_batAvg['Strike_rate'].mean():
        return 'High'
    elif x <= 0.75 * df_batAvg['Strike_rate'].mean():
        return 'Low'
    else:
        return 'Average'
    
def varlevel(x):
    if x <= 0.75*df_batAvg['Var'].mean():
        return 'High Consistency'
    elif x >= 1.25 * df_batAvg['Var'].mean():
        return 'Inconsistent'
    else:
        return 'Average'
    

df_batAvg['Runs_level'] = df_batAvg['AvgRuns_match'].map(lambda x: runlevel(x))
df_batAvg['Strike_level'] = df_batAvg['Strike_rate'].map(lambda x: strikelevel(x))
df_batAvg['Var_level'] = df_batAvg['Var'].map(lambda x: varlevel(x))


dfteam = df.groupby(['Batting_team','Bowling_team', 'Match_id', 'Innings', 'Over']).sum().reset_index()
over = 0
total = 0
Total_runs = []

for i in range(len(dfteam)):
    if dfteam.iloc[i]['Over'] >= over:
        total =+ total + dfteam.iloc[i]['Batsman_runs'] + dfteam.iloc[i]['Extra_runs']
        over =+ 1
        Total_runs.append(total)
    else:
        total = dfteam.iloc[i]['Batsman_runs'] + dfteam.iloc[i]['Extra_runs']
        Total_runs.append(total)
        over = 1


dfteam['Total_runs'] = pd.Series(Total_runs)



# ## Defining bowler metrics
# def caught(x):
#     if x == 'caught':
#         return 1
#     else:
#         return 0
# def bowled(x):
#     if x == bowled:
#         return 1
#     else:
#         return 0
# def runout(x):
#     if x == 'runout':
#         return 1
#     else:
#         return 0
# def lbw(x):
#     if x == 'lbw':
#         return 1
#     else:
#         return 0
# def stump(x):
#     if x == 'stump':
#         return 1
#     else:
#         return 0
# def candb(x):
#     if x == 'caught and bowled':
#         return 1
#     else:
#         return 0


# df['Caught'] = df['Dismisal_kind'].apply(lambda x: caught(x))
# df['Bowled'] = df['Dismisal_kind'].apply(lambda x: bowled(x))
# df['Lbw'] = df['Dismisal_kind'].apply(lambda x: lbw(x))
# df['Runout'] = df['Dismisal_kind'].apply(lambda x: runout(x))
# df['Stump'] = df['Dismisal_kind'].apply(lambda x: stump(x))
# df['Caught_bowled'] = df['Dismisal_kind'].apply(lambda x: candb(x))


# ## Creating a bowler dataframe
# df_bowl = pd.merge(df.groupby(['Bowler', 'Match_id']).sum().reset_index().filter(regex = "[Innings,Over,Ball]"), df.groupby(['Bowler', 'Match_id']).count().reset_index()[['Bowler','Match_id', 'Ball']].rename(columns = {'Ball': 'TotalDeliveries'}), on = ['Bowler', 'Match_id'])

# df_bowlAvg = pd.merge(df.groupby(['Bowler', 'Match_id']).sum().reset_index().filter(regex = "[^Match_id,Innings,Over,Ball]") ,  df.groupby(['Bowler', 'Match_id']).count().reset_index()[['Bowler', 'Ball']].rename(columns = {'Ball': 'Delieveries_bowled'}), on = 'Bowler').groupby('Bowler').mean().reset_index()


# df_bowl_Avg_merge = pd.merge(df_bowl.groupby('Bowler').sum().reset_index().filter(regex = '[^Match_id,Innings,Over,Ball]'), df_bowlAvg.filter(regex = "[^Caught,Bowled,Stump,Caught_bowled, Lbw, Runout]").rename(columns = {'Delieveries_bowled':'AvgDeliveries_bowled','Batsman_runs': 'AvgBatsman_runs', 'Extra_runs': 'AvgExtra_runs' , 'Six': 'AvgSix', 'Four': 'AvgFour', 'wicket' : 'AvgWicket'}),
#  on = 'Bowler')

# dfecon = df.groupby(['Bowler','Match_id', 'Over']).sum().reset_index().groupby('Bowler').mean().reset_index()[['Bowler','Batsman_runs', 'Extra_runs']]
# dfecon['Economy'] = dfecon['Batsman_runs'] + dfecon['Extra_runs']



# df_bowler = pd.merge(df_bowl_Avg_merge, dfecon[['Bowler','Economy']], on = 'Bowler')


# ## Bowler variance
# df_bowler['Runs_sq'] = 0
# df_bowler['Matches_played'] = 0
# for i in range(len(df_bowler)):
#     for j in range(len(df_bowl)):
#         if df_bowler.iloc[i]['Bowler'] == df_bowl.iloc[j]['Bowler']:
#             df_bowler['Runs_sq'].iloc[i] += (df_bowl.iloc[j]['Batsman_runs'] +  df_bowl.iloc[j]['Extra_runs'])**2
#             df_bowler['Matches_played'].iloc[i] += 1


# df_bowler['Var'] = (df_bowler['Runs_sq'] - (df_bowler['Matches_played']*((df_bowler['AvgBatsman_runs']+df_bowler['AvgExtra_runs'])**2)))/(df_bowler['Matches_played'] -1)

 

# df_bowler = pd.merge(df_bowler, df[['Bowler', 'Bowling_team']].drop_duplicates(keep = 'first'), on = 'Bowler')
# df_bowler.dropna(inplace = True)


# ##Bowler Scatter chart

# trace1 = go.Scatter(x = df_bowler['wicket'], y = df_bowler['Economy'], mode = 'markers', hoverinfo = 'text' , text = df_bowler['Bowler'] + "   " + df_bowler['Bowling_team'] , marker = dict(size = 25, symbol = 'circle' , cmin = 0, autocolorscale = True, reversescale = True, colorbar = {'title' : {'text': 'Bowler Variance'}}, color = df_bowler['Var']))

# layout = go.Layout(paper_bgcolor = 'white',plot_bgcolor = 'white' , hovermode = 'closest',  yaxis = dict(title = dict(text = "Economy" , font = dict(color ='black')) ,showgrid = True, zeroline = False, color = 'black'), xaxis = dict(title = dict( text = 'Total wickets'), showgrid = False, zeroline = False), title = dict(text = "All Bowler's Performance - Economy, Total Wickets, Variance"))


# data = [trace1]

# fig22 = go.Figure(data = data, layout = layout)

# plotly.offline.init_notebook_mode(connected=True)
# plotly.offline.plot(fig22, include_plotlyjs = False, output_type = 'div')


# ##Bowler histogram


# team_list = ['Peshawar Zalmi', 'Karachi Kings', 'Islamabad United', 'Quetta Gladiators', 'Lahore Qalandars']

# trace0 = go.Histogram(x= df_bowler.loc[df_bowler['Bowling_team'] == team_list[0], 'Economy'], opacity = 0.75, name = team_list[0])
# trace1 = go.Histogram(x= df_bowler.loc[df_bowler['Bowling_team'] == team_list[1], 'Economy'], opacity = 0.75, name = team_list[1])
# trace2 = go.Histogram(x= df_bowler.loc[df_bowler['Bowling_team'] == team_list[2], 'Economy'], opacity = 0.75, name = team_list[2])
# trace3 = go.Histogram(x= df_bowler.loc[df_bowler['Bowling_team'] == team_list[3], 'Economy'], opacity = 0.75, name = team_list[3])
# trace4 = go.Histogram(x= df_bowler.loc[df_bowler['Bowling_team'] == team_list[4], 'Economy'], opacity = 0.75, name = team_list[4])

# fig4 = plotly.tools.make_subplots(rows = 3, cols = 2)
# fig4.append_trace(trace0,1,1)
# fig4.append_trace(trace1,1,2)
# fig4.append_trace(trace2,2,1)
# fig4.append_trace(trace3,2,2)
# fig4.append_trace(trace4,3,1)

# fig4.layout.title = dict(text = "Distribution of average bowler economy for each team")

# plotly.offline.plot(fig4, include_plotlyjs = False, output_type = 'div')


# ##Setting up a ranking datafrane

# dfscore = df_batAvg[['Batsman_strike', 'Batting_team','AvgRuns_match', 'Strike_rate', 'Var']].rename(columns = {'Batsman_strike': 'Player', 'Batting_team':'Team','Var': 'Bat_var'}).merge(df_bowler[['Bowler', 'Economy', 'AvgWicket','AvgDeliveries_bowled', 'Var', 'Bowling_team']].rename(columns = {'Bowler':'Player','Var':'Bowl_var','Bowling_team':'Team'}), on = ['Player','Team'], how = 'outer').fillna(value = 0)                                                                                                          


# ##Player Ranking Metrics

# f = 'Strike_rate'
# g = 'AvgRuns_match'
# h = 'AvgWicket'


# Batsman = np.quantile(dfscore['AvgRuns_match'], 0.5)
# Batvarmean = dfscore.loc[dfscore['AvgRuns_match'] >= Batsman, "Bat_var"].mean()


# Bowlvarmean = dfscore.loc[dfscore['AvgDeliveries_bowled'] >= 0, "Bowl_var"].mean()

# dfscore['BatVarRank'] = np.zeros(len(dfscore))

# dfscore['BowlVarRank'] = np.zeros(len(dfscore))


# def greaterf(x):
#         if x == 0:
#             return 0
#         if x >= 1.25 * dfscore[f].mean():
#             return 3
#         if x<= 0.75 *dfscore[f].mean():
#             return  1
#         else:
#             return 2

# def greaterg(x):
#         if x == 0:
#             return 0
#         if x >= 1.25 * dfscore[g].mean():
#             return 3
#         if x<= 0.75 *dfscore[g].mean():
#             return  1
#         else:
#             return 2

# def greaterh(x):
#         if x == 0:
#             return 0
#         if x >= 1.25 * dfscore[h].mean():
#             return 3
#         if x<= 0.75 *dfscore[h].mean():
#             return  1
#         else:
#             return 2


# dfscore[f + str('Rank')] = dfscore[f].map(lambda x: greaterf(x))    
# dfscore[g + str('Rank')] = dfscore[g].map(lambda x: greaterg(x))
# dfscore[h + str('Rank')] = dfscore[h].map(lambda x: greaterh(x))

# def ecorank(x):
#     if x == 0:
#         return 0
#     elif x >= 1.25 * dfscore.loc[dfscore['AvgDeliveries_bowled'] >= 0, "Economy"].mean():
#         return 1
#     elif x <= 0.75 * dfscore.loc[dfscore['AvgDeliveries_bowled'] >= 0, "Economy"].mean():
#         return 3
#     else:
#         return 2
    
# dfscore['EconomyRank'] = dfscore['Economy'].map(lambda x: ecorank(x))


# for i in range(len(dfscore)):
#     if dfscore.iloc[i]['AvgRuns_match'] <= Batsman:
#         dfscore['BatVarRank'].iloc[i] = 0
#     elif dfscore['Bat_var'].iloc[i] >= (1.25 * Batvarmean):  
#         dfscore['BatVarRank'].iloc[i] = 1
#     elif dfscore.iloc[i]['Bat_var'] <= 0.75 * Bowlvarmean:
#         dfscore['BatVarRank'].iloc[i] = 3
#     else:
#         dfscore['BatVarRank'].iloc[i] = 2
        
        
        
# for i in range(len(dfscore)):
#     if dfscore.iloc[i]['AvgDeliveries_bowled'] == 0:
#         dfscore['BowlVarRank'].iloc[i] = 0
#     elif dfscore['Bowl_var'].iloc[i] >= (1.25 * Bowlvarmean):  
#         dfscore['BowlVarRank'].iloc[i] = 1
#     elif dfscore.iloc[i]['Bowl_var'] <= 0.75 * Bowlvarmean:
#         dfscore['BowlVarRank'].iloc[i] = 3
#     else:
#         dfscore['BowlVarRank'].iloc[i] = 2



# ##Creating PlayerScore column as the sum of all metrics

# dfscore['PlayerScore'] = dfscore['AvgRuns_matchRank'] + dfscore['Strike_rateRank'] + dfscore['AvgWicketRank'] + dfscore['BatVarRank'] + dfscore['BowlVarRank'] + dfscore['EconomyRank']


# ##Ranking levels

# def mvprank(x):
#     if x >= np.quantile(dfscore['PlayerScore'], 0.90):
#         return 4
#     elif x >= np.quantile(dfscore['PlayerScore'], 0.75):
#             return 3
#     elif x <= np.quantile(dfscore['PlayerScore'], 0.50):
#             return 1
#     else:
#         return 2
    
# dfscore['PlayerRank'] = dfscore['PlayerScore'].map(lambda x: mvprank(x))
# dfscore['MVP'] = np.zeros(len(dfscore))
# dfscore.loc[dfscore['PlayerRank']==4,'MVP'] = 'MVP'
# dfscore.loc[dfscore['PlayerRank']==3, 'MVP'] = 'VP'
# dfscore.loc[dfscore['PlayerRank'] <3, 'MVP'] = 'Below Average'


# ## Top 10 Players
# dfscore.sort_values(by = 'PlayerScore', ascending = False).head(10)


# #Boxplot

# trace0 = go.Box(y = dfscore.loc[dfscore['Team'] == 'Peshawar Zalmi', 'PlayerScore'], name = 'Peshawar Zalmi', boxmean = 'sd', boxpoints = 'all')
# trace1 = go.Box(y = dfscore.loc[dfscore['Team'] == 'Islamabad United', 'PlayerScore'], name = 'Islamabad United', boxmean = 'sd', boxpoints = 'all')
# trace2 = go.Box(y = dfscore.loc[dfscore['Team'] == 'Quetta Gladiators', 'PlayerScore'], name = 'Quetta Gladiators', boxmean = 'sd', boxpoints = 'all')
# trace3 = go.Box(y = dfscore.loc[dfscore['Team'] == 'Karachi Kings', 'PlayerScore'], name = 'Karachi Kings' , boxmean = 'sd', boxpoints = 'all')
# trace4 = go.Box(y = dfscore.loc[dfscore['Team'] == 'Lahore Qalandars', 'PlayerScore'], name = 'Lahore Qalandars' , boxmean = 'sd', boxpoints = 'all')

# data = [trace0, trace1, trace2, trace3, trace4]

# fig111 = go.Figure(data = data)
# fig111.layout.title = dict(text = "Player Score distribution for each team" )

# import bs4
# from bs4 import BeautifulSoup

# plotly.offline.plot(fig111, include_plotlyjs = False, output_type = 'div')


TypeError: plot() missing 1 required positional argument: 'kind'

In [21]:
import plotly.express as px

def wick(x):
    if x != "No dismisal":
        return 1
    else:
        return 0
df['wicket'] = df['Dismisal_kind'].apply(lambda x: wick(x))


def isSix(x):
    if x == 6:
        return 1
    else:
        return 0
    
def isFour(x):
    if x == 4:
        return 1
    else:
        return 0
df['Six'] = df['Batsman_runs'].map(lambda x: isSix(x))
df['Four'] = df['Batsman_runs'].map(lambda x: isFour(x))



#Ball-by-ball detail for each batsman

df_bat = pd.merge(df.groupby(['Batsman_strike','Match_id']).sum().reset_index()[['Batsman_strike','Match_id','Batsman_runs']], df.groupby(['Batsman_strike', 'Match_id']).count().reset_index()[['Batsman_strike','Match_id', 'Ball']].rename(columns = {'Ball': 'Balls_played'}), on = ['Batsman_strike', 'Match_id'])



##Batting datafrane
df_batAvg = pd.merge(pd.merge(df_bat.groupby('Batsman_strike').mean().reset_index()[['Batsman_strike','Batsman_runs', 'Balls_played']].rename(columns = {'Batsman_runs' : 'AvgRuns_match' , 'Balls_played': 'AvgBalls_match'}), df[['Batsman_strike', 'Batting_team']].drop_duplicates(keep = 'first'), on = 'Batsman_strike') , pd.merge(df.groupby(['Batsman_strike']).sum().reset_index()[['Batsman_strike', 'Batsman_runs', 'Six', 'Four']].rename(columns = {'Batsman_runs': 'TotalRuns', 'Six':'TotalSixes', 'Four':'TotalFours'}), df.groupby(['Batsman_strike']).count().reset_index()[['Batsman_strike' , 'Ball']].rename(columns = {'Ball': 'Balls_played'}), on ='Batsman_strike'), on = 'Batsman_strike')
                                                                                                                                     
                                                                                                                                     

df_batAvg['Runs_sq'] = 0
df_batAvg['Matches_played'] = 0


#To calculate variance
for i in range(len(df_batAvg)):
    for j in range(len(df_bat)):
        if df_batAvg.iloc[i]['Batsman_strike'] == df_bat.iloc[j]['Batsman_strike']:
            df_batAvg['Runs_sq'].iloc[i] += df_bat.iloc[j]['Batsman_runs'] **2
            df_batAvg['Matches_played'].iloc[i] += 1


#Strke rate, % boundary and Variance
df_batAvg['Strike_rate'] = df_batAvg['TotalRuns'] * 100 /df_batAvg['Balls_played']
df_batAvg['PercentageRuns_boundary'] = (df_batAvg['TotalSixes']* 6 + df_batAvg['TotalFours'] * 4) * 100/df_batAvg['TotalRuns']
df_batAvg["Var"] = (df_batAvg['Runs_sq'] - df_batAvg['Matches_played']*(df_batAvg['AvgRuns_match']**2))/(df_batAvg['Matches_played'] - 1)
df_batAvg = df_batAvg.sort_values(by = 'TotalRuns', ascending = False)


##Batsman Scatter

z = df_batAvg['Batsman_strike']
trace = go.Scatter(y = df_batAvg['Strike_rate'], x= df_batAvg['AvgRuns_match'], mode = 'markers', showlegend = False , text = df_batAvg['Batsman_strike'] + "  " + df_batAvg['Batting_team'], hoverinfo =  "text", marker = dict(size = 20, symbol = 'circle' , cmin = 0, autocolorscale = True, reversescale = True, colorbar = {'title' : {'text': 'Score variance'}}, color = df_batAvg['Var']))
data = [trace]

layout = go.Layout(paper_bgcolor = 'white',plot_bgcolor = 'white' , yaxis = dict(title = dict(text = "Strike rate" , font = dict(color ='black')) ,showgrid = True, zeroline = False, color = 'black'), xaxis = dict( title= dict( text = 'Average runs'), showgrid = False, zeroline= False), title = dict(text = "All batsman's performance - Average runs, Strike rate, Score variance"))
fig = go.Figure(data = data, layout = layout)
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.plot(fig, include_plotlyjs = False, output_type = 'div')

px.scatter(data, y = df_batAvg['Strike_rate'], x= df_batAvg['AvgRuns_match'])



##prints


In [22]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

##Batsman Histogram


team_list = list(df_batAvg['Batting_team'].unique())

trace0 = go.Histogram(x= df_batAvg.loc[df_batAvg['Batting_team'] == team_list[0], 'AvgRuns_match'], opacity = 0.75, name = team_list[0])
trace1 = go.Histogram(x= df_batAvg.loc[df_batAvg['Batting_team'] == team_list[1], 'AvgRuns_match'], opacity = 0.75, name = team_list[1])
trace2 = go.Histogram(x= df_batAvg.loc[df_batAvg['Batting_team'] == team_list[2], 'AvgRuns_match'], opacity = 0.75, name = team_list[2])
trace3 = go.Histogram(x= df_batAvg.loc[df_batAvg['Batting_team'] == team_list[3], 'AvgRuns_match'], opacity = 0.75, name = team_list[3])
trace4 = go.Histogram(x= df_batAvg.loc[df_batAvg['Batting_team'] == team_list[4], 'AvgRuns_match'], opacity = 0.75, name = team_list[4])

fig4 = make_subplots(rows =3, cols = 2)
fig4.append_trace(trace0,1,1)
fig4.append_trace(trace1,2,1)
fig4.append_trace(trace2,2,2)
fig4.append_trace(trace3,3,1)
fig4.append_trace(trace4,3,2)

fig4.layout.title = dict(text = 'Distribution of average batsman scores')
fig4.layout.xaxis = dict(tickmode = 'linear', dtick = 10)


plotly.offline.plot(fig4, include_plotlyjs = False, output_type = 'div')

'<div>                            <div id="29adecb3-81da-43b3-8174-fe1289e1fb34" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("29adecb3-81da-43b3-8174-fe1289e1fb34")) {                    Plotly.newPlot(                        "29adecb3-81da-43b3-8174-fe1289e1fb34",                        [{"name":"Peshawar Zalmi","opacity":0.75,"x":[32.09090909090909,22.125,29.166666666666668,17.0,22.666666666666668,13.333333333333334,18.8,16.2,13.5,7.0,10.333333333333334,6.2,6.5,4.6,5.0,1.0,1.0],"xaxis":"x","yaxis":"y","type":"histogram"},{"name":"Karachi Kings","opacity":0.75,"x":[29.1,21.5,20.2,22.333333333333332,17.77777777777778,11.9,10.375,6.0,9.5,4.0,10.0,2.5,0.0],"xaxis":"x3","yaxis":"y3","type":"histogram"},{"name":"Islamabad United","opacity":0.75,"x":[30.444444444444443,19.0,19.75,15.44444444

In [110]:
#Bowling
## Defining bowler metrics
def caught(x):
    if x == 'caught':
        return 1
    else:
        return 0
def bowled(x):
    if x == bowled:
        return 1
    else:
        return 0
def runout(x):
    if x == 'runout':
        return 1
    else:
        return 0
def lbw(x):
    if x == 'lbw':
        return 1
    else:
        return 0
def stump(x):
    if x == 'stump':
        return 1
    else:
        return 0
def candb(x):
    if x == 'caught and bowled':
        return 1
    else:
        return 0


df['Caught'] = df['Dismisal_kind'].apply(lambda x: caught(x))
df['Bowled'] = df['Dismisal_kind'].apply(lambda x: bowled(x))
df['Lbw'] = df['Dismisal_kind'].apply(lambda x: lbw(x))
df['Runout'] = df['Dismisal_kind'].apply(lambda x: runout(x))
df['Stump'] = df['Dismisal_kind'].apply(lambda x: stump(x))
df['Caught_bowled'] = df['Dismisal_kind'].apply(lambda x: candb(x))


## Creating a bowler dataframe
#By each match
df_bowl_match = pd.merge(
    df.groupby(['Bowler', 'Match_id']).sum().reset_index().filter(regex = "[Innings,Over,Ball]"), 
    df.groupby(['Bowler', 'Match_id']).count().reset_index()[['Bowler','Match_id', 'Ball']].rename(columns =        {'Ball': 'TotalDeliveries'}
    ), on = ['Bowler', 'Match_id'])

#Avg deliveries bye ach match
df_bowl_matchAvg = pd.merge(
    df.groupby(['Bowler', 'Match_id']).sum().reset_index().filter(regex = "[^Match_id,Innings,Over,Ball]") ,       df.groupby(['Bowler', 'Match_id']).count().reset_index()[['Bowler', 'Ball']].rename(columns = {'Ball': 'Delieveries_bowled'}), on = 'Bowler').groupby('Bowler').mean().reset_index()

#Avg tot runs by each match
df_bowl_Avg_Tot = pd.merge(
    df_bowl_match.groupby('Bowler').sum().reset_index().filter(regex = '[^Match_id,Innings,Over,Ball]'),           df_bowlAvg.filter(regex = "[^Caught,Bowled,Stump,Caught_bowled, Lbw, Runout]").rename(columns =      {'Delieveries_bowled':'AvgDeliveries_bowled','Batsman_runs': 'AvgBatsman_runs', 'Extra_runs': 'AvgExtra_runs' , 'Six': 'AvgSix', 'Four': 'AvgFour', 'wicket' : 'AvgWicket'}),
 on = 'Bowler')

#Bowler Economy
dfecon = df.groupby(['Bowler','Match_id', 'Over']).sum().reset_index().groupby('Bowler').mean().reset_index()[['Bowler','Batsman_runs', 'Extra_runs']]
dfecon['Economy'] = dfecon['Batsman_runs'] + dfecon['Extra_runs']

#Bowler final dataframe
df_bowler_summ = pd.merge(df_bowl_Avg_Tot.loc[df_bowl_Avg_Tot['TotalDeliveries'] > 50], dfecon[['Bowler','Economy']], on = 'Bowler')


## Bowler variance
df_bowler['Runs_sq'] = 0
df_bowler['Matches_played'] = 0
for i in range(len(df_bowler)):
    for j in range(len(df_bowl)):
        if df_bowler.iloc[i]['Bowler'] == df_bowl.iloc[j]['Bowler']:
            df_bowler['Runs_sq'].iloc[i] += (df_bowl.iloc[j]['Batsman_runs'] +  df_bowl.iloc[j]['Extra_runs'])**2
            df_bowler['Matches_played'].iloc[i] += 1


df_bowler['Var'] = (df_bowler['Runs_sq'] - (df_bowler['Matches_played']*((df_bowler['AvgBatsman_runs']+df_bowler['AvgExtra_runs'])**2)))/(df_bowler['Matches_played'] -1)

 

df_bowler = pd.merge(df_bowler_summ, df[['Bowler', 'Bowling_team']].drop_duplicates(keep = 'first'), on = 'Bowler')
df_bowler.dropna(inplace=True)
print('df bowler')
print(df_bowler)

df bowler
                Bowler  Batsman_runs  Extra_runs  wicket  Six  Four  Caught  \
0          Aamer Yamin            82           4       5    3     6       2   
1            Anwar Ali           272          10       6   15    21       5   
3       Bilawal Bhatti            93          13       1    2     9       1   
4            CJ Jordan           124          12       7    2    11       7   
5            DJG Sammy            96           9       2    4    10       2   
6           GD Elliott           112           3       5    5     9       2   
7            Hasan Ali           253           5      13    6    26       5   
8           Hasan Khan           213          19       9    9    14       3   
10          Imad Wasim           189           5       8    9    13       1   
11         Junaid Khan            96           5       3    7     9       2   
13         Mahmudullah           111           1       7    4    11       5   
14       Mohammad Amir           261      

In [112]:
# ##Bowler histogram


team_list = ['Peshawar Zalmi', 'Karachi Kings', 'Islamabad United', 'Quetta Gladiators', 'Lahore Qalandars']

trace0 = go.Histogram(x= df_bowler.loc[df_bowler['Bowling_team'] == team_list[0], 'Economy'], opacity = 0.75, name = team_list[0])
trace1 = go.Histogram(x= df_bowler.loc[df_bowler['Bowling_team'] == team_list[1], 'Economy'], opacity = 0.75, name = team_list[1])
trace2 = go.Histogram(x= df_bowler.loc[df_bowler['Bowling_team'] == team_list[2], 'Economy'], opacity = 0.75, name = team_list[2])
trace3 = go.Histogram(x= df_bowler.loc[df_bowler['Bowling_team'] == team_list[3], 'Economy'], opacity = 0.75, name = team_list[3])
trace4 = go.Histogram(x= df_bowler.loc[df_bowler['Bowling_team'] == team_list[4], 'Economy'], opacity = 0.75, name = team_list[4])

fig4 = make_subplots(rows =3, cols = 2)
fig4.append_trace(trace0,1,1)
fig4.append_trace(trace1,1,2)
fig4.append_trace(trace2,2,1)
fig4.append_trace(trace3,2,2)
fig4.append_trace(trace4,3,1)

fig4.layout.title = dict(text = "Distribution of average bowler economy for each team")

fig4.show()

# plotly.offline.plot(fig4, include_plotlyjs = False, output_type = 'div')


,Match_id,Innings,Over,Ball,Batting_team,Bowling_team,Batsman_strike,Non_striker,Batsman_runs,Extra_runs,...,Player_dismissed,wicket,Six,Four,Caught,Bowled,Lbw,Runout,Stump,Caught_bowled
Bowler,,,,,,,,,,,,,,,,,,,,,
Aamer Yamin,74,74,74,74,74,74,74,74,74,74,...,74,74,74,74,74,74,74,74,74,74
Abrar Ahmed,36,36,36,36,36,36,36,36,36,36,...,36,36,36,36,36,36,36,36,36,36
Amad Butt,31,31,31,31,31,31,31,31,31,31,...,31,31,31,31,31,31,31,31,31,31
Anwar Ali,188,188,188,188,188,176,188,188,188,188,...,188,188,188,188,188,188,188,188,188,188
BB McCullum,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
Bilawal Bhatti,98,98,98,98,98,98,98,98,98,98,...,98,98,98,98,98,98,98,98,98,98
CJ Jordan,120,120,120,120,120,120,120,120,120,120,...,120,120,120,120,120,120,120,120,120,120
CS Delport,11,11,11,11,11,11,11,11,11,11,...,11,11,11,11,11,11,11,11,11,11
DJG Sammy,63,63,63,63,63,63,63,63,63,63,...,63,63,63,63,63,63,63,63,63,63
